In [1]:
import numpy as np
import pandas as pd

### Pdf reading

In [2]:
%pip install --upgrade pdfplumber pdfminer -q

import pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [3]:
def _convert_pdf_to_text(path):
    """Convert the PDF to text and store it in the self.content
    attribute.
    """
    text = []
    pdf = pdfplumber.open(path)
    for page in range(len(pdf.pages)):
        # get the page object
        page_obj = pdf.pages[page]
        # extract text from the page
        text.append(page_obj.extract_text(x_tolerance=2, x_tolerance_ratio=None, y_tolerance=3, layout=False, x_density=7.25, y_density=13, line_dir_render=None, char_dir_render=None))
    text = "\n".join(text)
    
    return text

In [4]:
text = _convert_pdf_to_text("D:\Projects\LLM\Createdb\docs\\10.1.1.66.5347.pdf")

In [5]:
print(text)

Composite Structures 67 (2005) 239–249
www.elsevier.com/locate/compstruct
Structural damage identification in laminated structures
using FRF data
J.V. Arau´jo dos Santos, C.M. Mota Soares *, C.A. Mota Soares, N.M.M. Maia
IDMEC/IST, Instituto Superior Te´cnico, Av. Rovisco Pais, 1049-001 Lisbon, Portugal
Available online 5 November 2004
Abstract
A damage identification technique based on frequency response functions (FRF) sensitivities is presented. This technique leads to
a set of linear equations, which is solved using an algorithm that constrains the solution to be physically admissible. Damage sim-
ulation and identification on a laminated rectangular plate is performed. The influence of the number of natural frequencies and
mode shapes used on the FRF computation, as well as the frequency range, the excitation location and the number of measured
degrees of freedom (m-DOF) is studied. Numerical tests show that the best accuracy is obtained when using the dynamic expansion
of the m-D

In [6]:
data = [[1,text]]

In [7]:
df = pd.DataFrame(data, columns=['id','raw_text'])

In [8]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\sai18\AppData\Local\Temp\ipykernel_25100\2340421553.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


### Punctuation removal

In [9]:
import string
import re 
# import nltk
# from nltk.corpus import stopwords


data_file_path = "data.txt"
with open(data_file_path, "r", encoding="utf-8") as file:
    data_text = file.read()
    
words = []
word = ''


print(len(data_text))
#punctuation

def pc(data_text):
    data_text = data_text.split('\"' and "," and ":" and '\”' and '\“')
    text = ""
    for i in data_text:
        text = text + i
    print(len(text))

    text = ""

    for i in data_text:
        text = text + i
    
    return text

df['pc_removal'] = df['raw_text'].apply(pc)


0
37043


### url

In [10]:
# #url
url_pattern = r'https?:?//\S+'

def add_urls(text):
    httplinks = re.findall(url_pattern,text)
    return httplinks
url_pattern = re.compile(r'https?:?//\S+')
def remove_urls(text):
    return url_pattern.sub('', text)

df['urls'] = df['pc_removal'].apply(add_urls)
df['without_url'] = df['pc_removal'].apply(remove_urls)

### 

### split refrences

In [11]:
df['without_refs'] = df['without_url'][0].split('references')[0]
df['refs'] = df['without_url'][0].split('references')[1]

### numbers

In [12]:
import re

number_pattern = re.compile(r'\b(?<!\[)(-?\d+(?:\.\d+)?)\b(?![^\[]*\])')

def remove_number(text):
    return number_pattern.sub('', text)

# Assuming df['without_url'] is your DataFrame column containing text
df['without_numbers'] = df['without_refs'].apply(remove_number)


### mail

In [13]:
# mail
mail_pattern = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
def remove_mail(text):
    return mail_pattern.sub('', text)
df['without_mail'] = df['without_numbers'].apply(remove_mail)

### sentence segmentation

In [14]:
def sen_seg(data_text):
    sentences = []

    sentence = ''

    for i in range(len(data_text)):
        if data_text[i] == ".":
            sentences.append(sentence)
            sentence = ""
            continue
        else:
            sentence += data_text[i]

    sentences1 = []
    for i in sentences:
        if("\n" in i):
            # print("Yes")
            i = i.replace("\n", " ")
        # print(i)
        sentences1.append(i)
    
    return sentences1

df['sentences'] = df['without_mail'].apply(sen_seg)


In [15]:
print(len(df['sentences'][0]))

436


In [16]:
def checker(sentences):
    new = []
    for s in sentences:
        if(len(s) > 30):
            new.append(s)

    return new

df['sentences'] = df['sentences'].apply(checker)

## String Slicing

In [17]:
from nltk.tokenize import word_tokenize

new = word_tokenize(text)

In [18]:
# import numpy as np
# from wordfreq import get_frequency_dict

# word_prob = get_frequency_dict(lang='en', wordlist='large')
# max_word_len = max(map(len, word_prob))  # 34

# def str_slice(text):
#     def viterbi_segment(text, debug=False):
#         probs, lasts = [1.0], [0]
#         for i in range(1, len(text) + 1):
#             new_probs = []
#             for j in range(max(0, i - max_word_len), i):
#                 substring = text[j:i]
#                 length_reward = np.exp(len(substring))
#                 freq = word_prob.get(substring, 0) * length_reward
#                 compounded_prob = probs[j] * freq
#                 new_probs.append((compounded_prob, j))
                
#                 if debug:
#                     print(f'[{j}:{i}] = "{text[lasts[j]:j]} & {substring}" = ({probs[j]:.8f} & {freq:.8f}) = {compounded_prob:.8f}')

#             prob_k, k = max(new_probs)  # max of a touple is the max across the first elements, which is the max of the compounded probabilities
#             probs.append(prob_k)
#             lasts.append(k)

#             if debug:
#                 print(f'i = {i}, prob_k = {prob_k:.8f}, k = {k}, ({text[k:i]})\n')


#         # when text is a word that doesn't exist, the algorithm breaks it into individual letters.
#         # in that case, return the original word instead
#         if len(set(lasts)) == len(text):
#             return text

#         words = []
#         k = len(text)
#         while 0 < k:
#             word = text[lasts[k]:k]
#             words.append(word)
#             k = lasts[k]
#         words.reverse()
#         return ' '.join(words)

#     def split_message(message):
#         new_message = ' '.join(viterbi_segment(wordmash, debug=False) for wordmash in message.split())
#         return new_message
    
#     new_sentences = []

#     for s in text:
#         new_msg = split_message(s)
#         new_sentences.append(new_msg)

#     return new_sentences

In [19]:
# df['string_slicing'] = df['sentences'].apply(str_slice)

In [20]:
df['sentences'][0]

['com/locate/compstruct structural damage identification in laminated structures using frf data j',
 ' maia idmec/ist, instituto superior te´cnico, av',
 ' rovisco pais,  lisbon, portugal available online  november  abstract a damage identification technique based on frequency response functions (frf) sensitivities is presented',
 ' this technique leads to a set of linear equations, which is solved using an algorithm that constrains the solution to be physically admissible',
 ' damage sim- ulation and identification on a laminated rectangular plate is performed',
 ' the influence of the number of natural frequencies and mode shapes used on the frf computation, as well as the frequency range, the excitation location and the number of measured degrees of freedom (m-dof) is studied',
 ' numerical tests show that the best accuracy is obtained when using the dynamic expansion of the m-dof',
 ' it is also demonstrated that for small damage the errors are the main influence, whereas for large